# input

In [ ]:
# input txt
def readTxt():
    data = []
    with open("../data/R15.txt","r") as f:
        for line in f.readlines():
            line = line.strip("\n")
            line = line.split()
            data.append([float(i) for i in line])   
    return data
df = readTxt()

In [ ]:
# input csv
import pandas as pd
df = pd.read_csv('../data/spiral_312p_2d_3c.csv', header=None)

In [ ]:
import numpy as np
df = np.array(df, dtype = np.float32)
df_square = np.sum(df ** 2, axis=1, keepdims=True)
dist_square = df_square - 2 * df.dot(df.T) + df_square.T
dist_square = np.where(dist_square < 0, 0, dist_square) # to avoid negative float like -10^(-5)
dist = np.sqrt(dist_square)

# super-parameter d_c

In [ ]:
dist_flatten = dist.flatten()
dist_flatten = np.sort(dist_flatten)
N = len(df)
percentage = 0.02 # super-parameter
d_c = dist_flatten[round(N**2*percentage)]

Tune:
* 论文中推荐1%-2%，我发现2%可用于D31、R15，看来2%的确是一个还不错的值。
* Aggregation数据集需要调调参，我调了几次，发现4%比较合适。（这个算法的确帅气、优美、简洁、普适，我调了几下就调出来了）

In [ ]:
# def test_d_c(d_c, N):
#     n = 0
#     for i in range(N):
#         x = dist[i,:]
#         n = n + sum( x - d_c < 0)
#     n = n / N
#     if n > 0.01 * N and n < 0.02 * N:
#         print("OK")
#     else:
#         print("not OK")
#     return n
# n = test_d_c(d_c, N)

# calculate $\rho$

In [ ]:
# step kernal
# rou = np.zeros([N,2])
# rou[:,0] = np.arange(0, N)
# for i in range(N):
#     x = dist[i,:]
#     rou[i,1] = sum( x - d_c < 0)
# temp = np.lexsort(-rou.T)
# rou = rou[temp,:]

In [ ]:
# Gauss kernal
import math
rho_Gauss = np.zeros([N,2])
rho_Gauss[:,0] = np.arange(0, N)
for i in range(N-1):
    for j in range(i+1,N):
        rho_Gauss[i,1] = rho_Gauss[i,1] + math.exp(-(dist[i,j]/d_c)**2)
        rho_Gauss[j,1] = rho_Gauss[j,1] + math.exp(-(dist[i,j]/d_c)**2)
rho_Gauss = rho_Gauss[np.lexsort(-rho_Gauss.T),:]

# calculate $\delta$

In [ ]:
delta = np.zeros([N,2])
delta[:,0] = rho_Gauss[:,0]
for i in range(1,N):
    d = np.zeros(i)
    for j in range(i):        
        d[j] = dist[int(rho_Gauss[i,0]),int(rho_Gauss[j,0])]
    delta[i,1] = min(d)
delta[0,1] = max(delta[:,1])

# plot delta~rho

In [ ]:
import matplotlib.pyplot as plt
x = rho_Gauss[:,1]
y = delta[:,1]
fig = plt.figure() 
ax1 = fig.add_subplot(111)
plt.xlabel('rou') 
plt.ylabel('delta') 
ax1.scatter(x, y, marker='o') 
plt.show()

# choose the cluster centers

In [ ]:
# method1: use rho*delta
point = np.zeros([N,4])
point[:,0:2] = rho_Gauss
point[:,2] = delta[:,1]
point[:,3] = rho_Gauss[:,1] * delta[:,1]
temp = np.lexsort(-point.T)
point = point[temp,:]

## plot $\rho \cdot \delta$ ~ number

In [ ]:
import matplotlib.pyplot as plt
x_2 = np.arange(0, N) # 0, 1, 2 ... 599
y_2 = point[:,3]
fig = plt.figure() 
ax1 = fig.add_subplot(111)
plt.xlabel('point number') 
plt.ylabel('rou*delta') 
ax1.scatter(x_2, y_2, marker='o') 
plt.show()

## get the cluster centers

In [ ]:
step = np.quantile(point[:,3], 0.5) # use quantile to avoid another super-parameter
for i in range(1, N):
    z = np.logical_and(point[:,3] > step*i, point[:,3] <= step*(i+1))
    if sum(z) == 0:
        break
number_of_cluster = sum(point[:,3] > step*i)
id_of_cluster_center = point[0:number_of_cluster,0]
id_of_cluster_center = id_of_cluster_center.astype(int)

other paussible method to choose the cluster centers
* method2: use 2 percentage as threshold
* method3: use cluster method or SVM in delta~rho or rou*delta~t
* method4: let the user draw a rectangular (the original paper use this straightforward but subjective method)

# plot cluster graph

In [ ]:
# plot the cluster center in red and others in blue
x = df[:,0]
y = df[:,1]
fig = plt.figure() 
ax1 = fig.add_subplot(111)
plt.xlabel('X') 
plt.ylabel('Y') 
ax1.set_title('Scatter Plot') 
ax1.scatter(x, y, marker='o') 
x_center = df[id_of_cluster_center,0]
y_center = df[id_of_cluster_center,1]
ax1.scatter(x_center, y_center, c='r', marker='o')
plt.show() 

each remaining point is assigned to the same cluster as its nearest neighbor of higher density

column of point:
* c0 is id
* c1 is rho
* c2 is delta
* c3 is rho*delta
* c4 is cluster label

In [ ]:
# add a new column for cluster label
point = np.append(point, np.zeros([N,1]), axis=1)

# put the cluster center into each cluster
for i in range(number_of_cluster):
    point[i,4] = i + 1

# descend points by id
point_descend_by_id = point[np.argsort(point[:, 0])]

# flag for dist
is_dist = dist <= d_c * 5 # I use 5 to make sure that I can find a neighbor with higher density. # 5 is kind of super-parameter, can be 100 too.

In [ ]:
def find_the_nearest_neighbor_with_higher_density(i):

    global point_descend_by_id

    # if this point already has id
    if point_descend_by_id[i,4] != 0:
        return
    
    flag_neighbor = is_dist[i,:]
    dist_neighbor = dist[i,:][flag_neighbor]
    neighbor = point_descend_by_id[flag_neighbor]
    neighbor = np.append(neighbor, dist_neighbor[:,np.newaxis], axis=1)
    neighbor = neighbor[neighbor[:,1] > point_descend_by_id[i,1] + 10**(-4)] # 10**(-4) is used to avoid include itself
    neighbor = neighbor[np.argsort(neighbor[:, 5])]
    if neighbor[0,4] != 0:
        point_descend_by_id[i,4] = neighbor[0,4]
    else:
        find_the_nearest_neighbor_with_higher_density(int(neighbor[0,0])) # recursion
        point_descend_by_id[i,4] = point_descend_by_id[int(neighbor[0,0]),4]
for i in range(N):
    find_the_nearest_neighbor_with_higher_density(i)

In [ ]:
fig = plt.figure() 
ax1 = fig.add_subplot(111)
plt.xlabel('X') 
plt.ylabel('Y') 
ax1.set_title('Scatter Plot')
ax1.scatter(df[:,0], df[:,1], c = point_descend_by_id[:,4], cmap = "tab20")

# Davies-Bouldin Index 

In [ ]:
from sklearn import metrics
metrics.davies_bouldin_score(df, point_descend_by_id[:,4])

* Aggregation: 0.5036
* R15: 0.3148
* D31: 0.5519

# conclusion
* this algorithm
  * 该算法帅气优雅简洁（自不必说）、普适（可以用于球形和非球形数据集），是我们物理学家会喜欢的算法。
  * 该算法结合了K-means和DBSCAN的优点
* Further improvement
  * 光晕点
    * 原论文中给出了光晕点的定义，这个定义自然地给出了寻找方法
    * 所有的光晕点为新的一类，用一种新的颜色表示
  * 离群点
    * 原论文中把$\rho$小$\delta$大的点定义为离群点
    * 我们也可以把在$5d_c$中都没有更大密度邻居的点定义为离群点
    * 所有的离群点也可以被划分为新的一类，用一种新的颜色表示